Previously we have talked about using dimensionality technique for data processing/feature selection. In this notebook we will see an example of such workflow, using the store sales forecast example 

### Load feature engineered datasets

We have created lots of time related features before, let's reload the dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("../data/store/sales_processed.csv")

In [3]:
df.head()

,date,sales_log,onpromotion_log,is_wknd,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,...,store_nbr_1,store_nbr_2,store_nbr_3,store_nbr_4,store_nbr_5,store_nbr_6,store_nbr_7,store_nbr_8,store_nbr_9,store_nbr_10
0,2013-01-01,0.000000,0.0,0,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,2013-01-02,1.098612,0.0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2013-01-03,0.000000,0.0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,2013-01-04,1.386294,0.0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,2013-01-05,1.386294,0.0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


### Train - valid split

For time series related predictions, make sure to split train-valid by their time (avoid data leakage)

In [4]:
train = df[df['date'] < '2017-01-01']
valid = df[df['date'] >= '2017-01-01']

In [5]:
train = train.drop(['date'], axis=1)
valid = valid.drop(['date'], axis=1)

In [6]:
train_y = train['sales_log']
train_X = train.drop(['sales_log'], axis=1)

valid_y = valid['sales_log']
valid_X = valid.drop(['sales_log'], axis=1)

In [7]:
train_y.shape, train_X.shape, valid_y.shape, valid_X.shape

((43710,), (43710, 55), (6810,), (6810, 55))

### Perform dimensionality reduction

In [8]:
pca = PCA(n_components=10)

In [9]:
new_train_features = pca.fit_transform(train_X)

The 10 features captures 80% of variance of the original dataset (information)

In [10]:
(pca.explained_variance_ratio_).sum()

0.8021001768316256

### Train a linear model

In [11]:
linear_model = LinearRegression()
linear_model.fit(new_train_features, train_y)

LinearRegression()

Last time, in notebook 06, we have seen that by throwing all features into linear regression, we are seeing severe overfitting and most of the coefficients have extreme high values. Here we can see using the 10 new features, the coefficients are more well behaved

In [12]:
linear_model.coef_

array([ 9.44882438e-01, -7.22625311e-02,  1.97460204e-01,  2.40064147e-03,
        1.51658211e-03, -2.76771424e-04,  2.69603139e-02,  2.83471510e-02,
       -4.12241914e-03, -3.54922333e-03])

In [13]:
train_pred = linear_model.predict(new_train_features)

In [14]:
train_loss = np.sqrt(mean_squared_error(train_y, train_pred))
print(train_loss)

0.7305760028995053


### Making predictions on valid dataset

First, we have to also use the PCA model trained on the training dataset to tranform our valid dataset

In [15]:
new_valid_features = pca.transform(valid_X) ## note we're only using transform here, not fit_transform

In [16]:
y_pred = linear_model.predict(new_valid_features)

In [17]:
valid_loss = np.sqrt(mean_squared_error(valid_y, y_pred))
print(f'{valid_loss:.4f}')

0.7345


Boom! We can see that compared to notebook 06 (whose validation loss was an astronomical number), we are seeing by adding an additional dimensionality reduction step and using the new features, we can increase the power of the simple linear regression a lot.

### Scikit-learn pipeline

As you can see, we have two model training steps (PCA and linear regression), and we called the train and transform/predict twice for each train and valid dataset. To deal with these sequential training/processing needs, Scikit-learn has a class called Pipeline that make this process more steamlined 

In [18]:
from sklearn.pipeline import Pipeline

In [19]:
pipe = Pipeline([('pca', PCA(n_components=10)), ('linear_model', LinearRegression())])

In [20]:
pipe.fit(train_X, train_y)

Pipeline(steps=[('pca', PCA(n_components=10)),
                ('linear_model', LinearRegression())])

In [25]:
y_pred = pipe.predict(valid_X)
loss = np.sqrt(mean_squared_error(valid_y, y_pred))
print(f'{loss:.4f}')

0.7345


### Interpretability

Previously, we have seen that the sales_lag feature was really important (which makes sense). Here we can also take a look at the model coefficients:

In [26]:
linear_model.coef_

array([ 9.44882438e-01, -7.22625311e-02,  1.97460204e-01,  2.40064147e-03,
        1.51658211e-03, -2.76771424e-04,  2.69603139e-02,  2.83471510e-02,
       -4.12241914e-03, -3.54922333e-03])

The first variable has the highest importance by having the largest weight. However, how do we interpret the result?

In [27]:
pca_feature_explainability = pd.DataFrame(pca.components_.transpose(), 
                                          columns=[f'column_{i}' for i in range(1, 11)],
                                          index=train_X.columns)

In [28]:
pca_feature_explainability['column_1'].sort_values(ascending=False)[:5]

sales_lag          0.973549
onpromotion_log    0.171278
family_CLEANING    0.090983
year_2016          0.031679
quarter_4          0.014804
Name: column_1, dtype: float64

We can see that in the new feature, the first column (feature) is mostly comprised of sales_lag and onpromotion_log information. This suggest sales_lag still is most important indicator in the new model. (It is the most important contributor to the most important feature in the new model)

### HW

1. Do you think dimensionality_reduction can also improve performance for Ridge, Lasso and RandomForestRegression? Why or why not? 
2. Repeat the training process using Ridge, Lasso and RandomForestRegression. Try different hyperparameters and compare their performance.